# Building the Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
from IPython.display import clear_output
import time
!add-apt-repository --yes ppa:deadsnakes/ppa
!apt-get update
!apt-get install python3.6
!apt-get install python3.6-dev

!wget https://bootstrap.pypa.io/get-pip.py && python3.6 get-pip.py

import sys

sys.path[2] = '/usr/lib/python36.zip'
sys.path[3] = '/usr/lib/python3.6'
sys.path[4] = '/usr/lib/python3.6/lib-dynload'
sys.path[5] = '/usr/local/lib/python3.6/dist-packages'
sys.path[7] = '/usr/local/lib/python3.6/dist-packages/IPython/extensions'

!pip install tensorflow==1.12

clear_output()
time.sleep(1)

import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install tensorflow==1.12
!pip install keras==2.2.5

ERROR: Could not find a version that satisfies the requirement tensorflow==1.12 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==1.12


- Imports

In [ ]:
from collections import Counter
from sklearn.linear_model import LinearRegression

import os
import numpy as np
from numpy.random import seed
seed(1)
#from tensorflow import set_random_seed
#set_random_seed(2)


#from nfp.preprocessing import MolPreprocessor, GraphSequence

import gzip
import pickle
import pandas as pd

# Define Keras model
import keras
import keras.backend as K

from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

from keras.layers import (Input, Embedding, Dense, BatchNormalization, Dropout,
                                 Concatenate, Multiply, Add)

from keras.models import Model, load_model

#from nfp.layers import (MessageLayer, GRUStep, Squeeze, EdgeNetwork,
 #                              ReduceAtomToMol, ReduceBondToAtom,
  #                             GatherAtomToBond, ReduceAtomToPro)
#from nfp.models import GraphModel
import argparse
!pip install nfp

from nfp.preprocessing import features
from nfp.preprocessing import Tokenizer
from tqdm import tqdm
import time
!pip install rdkit-pypi
!pip install --upgrade rdkit-pypi pandas

from rdkit import Chem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


- Getting classes from the CASCADE paper

In [ ]:
def atomic_number_tokenizer(atom):
    return atom.GetAtomicNum()
def Mol_iter(df):
    for index,r in df.iterrows():
        yield(r['Mol'], r['atom_index'])

class SmilesPreprocessor(object):
    """ Given a list of SMILES strings, encode these molecules as atom and
    connectivity feature matricies.

    Example:
    >>> preprocessor = SmilesPreprocessor(explicit_hs=False)
    >>> inputs = preprocessor.fit(data.smiles)
    """

    def __init__(self, explicit_hs=True, atom_features=None, bond_features=None):
        """

        explicit_hs : bool
            whether to tell RDkit to add H's to a molecule.
        atom_features : function
            A function applied to an rdkit.Atom that returns some
            representation (i.e., string, integer) for the Tokenizer class.
        bond_features : function
            A function applied to an rdkit Bond to return some description.

        """

        self.atom_tokenizer = Tokenizer()
        self.bond_tokenizer = Tokenizer()
        self.explicit_hs = explicit_hs

        if atom_features is None:
            atom_features = features.atom_features_v1

        if bond_features is None:
            bond_features = features.bond_features_v1

        self.atom_features = atom_features
        self.bond_features = bond_features


    def fit(self, smiles_iterator):
        """ Fit an iterator of SMILES strings, creating new atom and bond
        tokens for unseen molecules. Returns a dictionary with 'atom' and
        'connectivity' entries """
        return list(self.preprocess(smiles_iterator, train=True))


    def predict(self, smiles_iterator):
        """ Uses previously determined atom and bond tokens to convert a SMILES
        iterator into 'atom' and 'connectivity' matrices. Ensures that atom and
        bond classes commute with previously determined results. """
        return list(self.preprocess(smiles_iterator, train=False))


    def preprocess(self, smiles_iterator, train=True):

        self.atom_tokenizer.train = train
        self.bond_tokenizer.train = train

        for smiles in tqdm(smiles_iterator):
            yield self.construct_feature_matrices(smiles)


    @property
    def atom_classes(self):
        """ The number of atom types found (includes the 0 null-atom type) """
        return self.atom_tokenizer.num_classes + 1


    @property
    def bond_classes(self):
        """ The number of bond types found (includes the 0 null-bond type) """
        return self.bond_tokenizer.num_classes + 1


    def construct_feature_matrices(self, smiles):
        """ construct a molecule from the given smiles string and return atom
        and bond classes.

        Returns
        dict with entries
        'n_atom' : number of atoms in the molecule
        'n_bond' : number of bonds in the molecule
        'atom' : (n_atom,) length list of atom classes
        'bond' : (n_bond,) list of bond classes
        'connectivity' : (n_bond, 2) array of source atom, target atom pairs.

        """

        mol = MolFromSmiles(smiles)
        if self.explicit_hs:
            mol = AddHs(mol)

        n_atom = len(mol.GetAtoms())
        n_bond = 2 * len(mol.GetBonds())

        # If its an isolated atom, add a self-link
        if n_bond == 0:
            n_bond = 1

        atom_feature_matrix = np.zeros(n_atom, dtype='int')
        bond_feature_matrix = np.zeros(n_bond, dtype='int')
        connectivity = np.zeros((n_bond, 2), dtype='int')

        bond_index = 0

        atom_seq = mol.GetAtoms()
        atoms = [atom_seq[i] for i in range(n_atom)]

        for n, atom in enumerate(atoms):

            # Atom Classes
            atom_feature_matrix[n] = self.atom_tokenizer(
                self.atom_features(atom))

            start_index = atom.GetIdx()

            for bond in atom.GetBonds():
                # Is the bond pointing at the target atom
                rev = bond.GetBeginAtomIdx() != start_index

                # Bond Classes
                bond_feature_matrix[n] = self.bond_tokenizer(
                    self.bond_features(bond, flipped=rev))

                # Connectivity
                if not rev:  # Original direction
                    connectivity[bond_index, 0] = bond.GetBeginAtomIdx()
                    connectivity[bond_index, 1] = bond.GetEndAtomIdx()

                else:  # Reversed
                    connectivity[bond_index, 0] = bond.GetEndAtomIdx()
                    connectivity[bond_index, 1] = bond.GetBeginAtomIdx()

                bond_index += 1


        return {
            'n_atom': n_atom,
            'n_bond': n_bond,
            'atom': atom_feature_matrix,
            'bond': bond_feature_matrix,
            'connectivity': connectivity,
        }


class ConnectivityAPreprocessor(object):
    """ Given a list of SMILES strings, encode these molecules as atom and
    connectivity feature matricies.

    Example:
    >>> preprocessor = SmilesPreprocessor(explicit_hs=False)
    >>> inputs = preprocessor.fit(data.smiles)
    """

    def __init__(self, explicit_hs=True, atom_features=None, bond_features=None):
        """

        explicit_hs : bool
            whether to tell RDkit to add H's to a molecule.
        atom_features : function
            A function applied to an rdkit.Atom that returns some
            representation (i.e., string, integer) for the Tokenizer class.
        bond_features : function
            A function applied to an rdkit Bond to return some description.

        """

        self.atom_tokenizer = Tokenizer()
        self.bond_tokenizer = Tokenizer()
        self.explicit_hs = explicit_hs

        if atom_features is None:
            atom_features = features.atom_features_v1

        if bond_features is None:
            bond_features = features.bond_features_v1

        self.atom_features = atom_features
        self.bond_features = bond_features


    def fit(self, smiles_iterator):
        """ Fit an iterator of SMILES strings, creating new atom and bond
        tokens for unseen molecules. Returns a dictionary with 'atom' and
        'connectivity' entries """
        return list(self.preprocess(smiles_iterator, train=True))


    def predict(self, smiles_iterator):
        """ Uses previously determined atom and bond tokens to convert a SMILES
        iterator into 'atom' and 'connectivity' matrices. Ensures that atom and
        bond classes commute with previously determined results. """
        return list(self.preprocess(smiles_iterator, train=False))


    def preprocess(self, smiles_iterator, train=True):

        self.atom_tokenizer.train = train
        self.bond_tokenizer.train = train

        for smiles in tqdm(smiles_iterator):
            yield self.construct_feature_matrices(smiles)


    @property
    def atom_classes(self):
        """ The number of atom types found (includes the 0 null-atom type) """
        return self.atom_tokenizer.num_classes + 1


    @property
    def bond_classes(self):
        """ The number of bond types found (includes the 0 null-bond type) """
        return self.bond_tokenizer.num_classes + 1


    def construct_feature_matrices(self, smiles):
        """ construct a molecule from the given smiles string and return atom
        and bond classes.

        Returns
        dict with entries
        'n_atom' : number of atoms in the molecule
        'n_bond' : number of bonds in the molecule
        'atom' : (n_atom,) length list of atom classes
        'bond' : (n_bond,) list of bond classes
        'connectivity' : (n_bond, 2) array of source atom, target atom pairs.

        """

        mol = MolFromSmiles(smiles)
        if self.explicit_hs:
            mol = AddHs(mol)

        n_atom = len(mol.GetAtoms())
        n_bond = 2 * len(mol.GetBonds())

        # If its an isolated atom, add a self-link
        if n_bond == 0:
            n_bond = 1

        atom_feature_matrix = np.zeros(n_atom, dtype='int')
        bond_feature_matrix = np.zeros(n_bond, dtype='int')
        connectivity = np.zeros((n_bond, 2), dtype='int')

        bond_index = 0

        atom_seq = mol.GetAtoms()
        atoms = [atom_seq[i] for i in range(n_atom)]

        for n, atom in enumerate(atoms):

            # Atom Classes
            atom_feature_matrix[n] = self.atom_tokenizer(
                self.atom_features(atom))

            start_index = atom.GetIdx()

            for bond in atom.GetBonds():
                # Is the bond pointing at the target atom
                rev = bond.GetBeginAtomIdx() != start_index

                # Bond Classes
                bond_feature_matrix[n] = self.bond_tokenizer(
                    self.bond_features(bond, flipped=rev))

                # Connectivity
                if not rev:  # Original direction
                    connectivity[bond_index, 0] = bond.GetBeginAtomIdx()
                    connectivity[bond_index, 1] = bond.GetEndAtomIdx()

                else:  # Reversed
                    connectivity[bond_index, 0] = bond.GetEndAtomIdx()
                    connectivity[bond_index, 1] = bond.GetBeginAtomIdx()

                bond_index += 1

        return {
            'n_atom': n_atom,
            'n_bond': n_bond,
            'atom': atom_feature_matrix,
            'bond': bond_feature_matrix,
            'connectivity': connectivity,
        }


class MolPreprocessor(SmilesPreprocessor):
    """ I should refactor this into a base class and separate
    SmilesPreprocessor classes. But the idea is that we only need to redefine
    the `construct_feature_matrices` method to have a working preprocessor that
    handles 3D structures.

    We'll pass an iterator of mol objects instead of SMILES strings this time,
    though.

    """

    def __init__(self, n_neighbors, cutoff, **kwargs):
        """ A preprocessor class that also returns distances between
        neighboring atoms. Adds edges for non-bonded atoms to include a maximum
        of n_neighbors around each atom """

        self.n_neighbors = n_neighbors
        self.cutoff = cutoff
        super(MolPreprocessor, self).__init__(**kwargs)


    def construct_feature_matrices(self, mol):
        """ Given an rdkit mol, return atom feature matrices, bond feature
        matrices, and connectivity matrices.

        Returns
        dict with entries
        'n_atom' : number of atoms in the molecule
        'n_bond' : number of edges (likely n_atom * n_neighbors)
        'atom' : (n_atom,) length list of atom classes
        'bond' : (n_bond,) list of bond classes. 0 for no bond
        'distance' : (n_bond,) list of bond distances
        'connectivity' : (n_bond, 2) array of source atom, target atom pairs.

        """

        n_atom = len(mol.GetAtoms())

        # n_bond is actually the number of atom-atom pairs, so this is defined
        # by the number of neighbors for each atom.
        #if there is cutoff,
        distance_matrix = Chem.Get3DDistanceMatrix(mol)

        if self.n_neighbors <= (n_atom - 1):
            n_bond = self.n_neighbors * n_atom
        else:
            # If there are fewer atoms than n_neighbors, all atoms will be
            # connected
            n_bond = distance_matrix[(distance_matrix < self.cutoff) & (distance_matrix != 0)].size

        if n_bond == 0: n_bond = 1

        # Initialize the matrices to be filled in during the following loop.
        atom_feature_matrix = np.zeros(n_atom, dtype='int')
        bond_feature_matrix = np.zeros(n_bond, dtype='int')
        bond_distance_matrix = np.zeros(n_bond, dtype=np.float32)
        connectivity = np.zeros((n_bond, 2), dtype='int')

        # Hopefully we've filtered out all problem mols by now.
        if mol is None:
            raise RuntimeError("Issue in loading mol")

        # Get a list of the atoms in the molecule.
        atom_seq = mol.GetAtoms()
        atoms = [atom_seq[i] for i in range(n_atom)]

        # Here we loop over each atom, and the inner loop iterates over each
        # neighbor of the current atom.
        bond_index = 0  # keep track of our current bond.
        for n, atom in enumerate(atoms):

            # update atom feature matrix
            atom_feature_matrix[n] = self.atom_tokenizer(
                self.atom_features(atom))

            # if n_neighbors is greater than total atoms, then each atom is a
            # neighbor.
            if (self.n_neighbors + 1) > len(mol.GetAtoms()):
                neighbor_end_index = len(mol.GetAtoms())
            else:
                neighbor_end_index = (self.n_neighbors + 1)

            distance_atom = distance_matrix[n, :]
            cutoff_end_index = distance_atom[distance_atom < self.cutoff].size

            end_index = min(neighbor_end_index, cutoff_end_index)

            # Loop over each of the nearest neighbors

            neighbor_inds = distance_matrix[n, :].argsort()[1:end_index]
            if len(neighbor_inds)==0: neighbor_inds = [n]
            for neighbor in neighbor_inds:

                # update bond feature matrix
                bond = mol.GetBondBetweenAtoms(n, int(neighbor))
                if bond is None:
                    bond_feature_matrix[bond_index] = 0
                else:
                    rev = False if bond.GetBeginAtomIdx() == n else True
                    bond_feature_matrix[bond_index] = self.bond_tokenizer(
                        self.bond_features(bond, flipped=rev))

                distance = distance_matrix[n, neighbor]
                bond_distance_matrix[bond_index] = distance

                # update connectivity matrix
                connectivity[bond_index, 0] = n
                connectivity[bond_index, 1] = neighbor

                bond_index += 1
        print(connectivity)

        return {
            'n_atom': n_atom,
            'n_bond': n_bond,
            'atom': atom_feature_matrix,
            'bond': bond_feature_matrix,
            'distance': bond_distance_matrix,
            'connectivity': connectivity,
        }


class MolBPreprocessor(MolPreprocessor):
    """
    This is a subclass of Molpreprocessor that preprocessor molecule with
    bond property target
    """
    def __init__(self, **kwargs):
        """
        A preprocessor class that also returns bond_target_matrix, besides the bond matrix
        returned by MolPreprocessor. The bond_target_matrix is then used as ref to reduce molecule
        to bond property
        """
        super(MolBPreprocessor, self).__init__(**kwargs)

    def construct_feature_matrices(self, entry):
        """
        Given an entry contining rdkit molecule, bond_index and for the target property,
        return atom
        feature matrices, bond feature matrices, distance matrices, connectivity matrices and bond
        ref matrices.

        returns
        dict with entries
        see MolPreproccessor
        'bond_index' : ref array to the bond index
        """
        mol, bond_index_array = entry

        n_atom = len(mol.GetAtoms())
        n_pro = len(bond_index_array)

        # n_bond is actually the number of atom-atom pairs, so this is defined
        # by the number of neighbors for each atom.
        #if there is cutoff,
        distance_matrix = Chem.Get3DDistanceMatrix(mol)

        if self.n_neighbors <= (n_atom - 1):
            n_bond = self.n_neighbors * n_atom
        else:
            # If there are fewer atoms than n_neighbors, all atoms will be
            # connected
            n_bond = distance_matrix[(distance_matrix < self.cutoff) & (distance_matrix != 0)].size

        if n_bond == 0: n_bond = 1

        # Initialize the matrices to be filled in during the following loop.
        atom_feature_matrix = np.zeros(n_atom, dtype='int')
        bond_feature_matrix = np.zeros(n_bond, dtype='int')
        bond_distance_matrix = np.zeros(n_bond, dtype=np.float32)
        bond_index_matrix = np.full(n_bond, -1, dtype='int')
        connectivity = np.zeros((n_bond, 2), dtype='int')

        # Hopefully we've filtered out all problem mols by now.
        if mol is None:
            raise RuntimeError("Issue in loading mol")

        # Get a list of the atoms in the molecule.
        atom_seq = mol.GetAtoms()
        atoms = [atom_seq[i] for i in range(n_atom)]

        # Here we loop over each atom, and the inner loop iterates over each
        # neighbor of the current atom.
        bond_index = 0  # keep track of our current bond.
        for n, atom in enumerate(atoms):
            # update atom feature matrix
            atom_feature_matrix[n] = self.atom_tokenizer(
                self.atom_features(atom))

            # if n_neighbors is greater than total atoms, then each atom is a
            # neighbor.
            if (self.n_neighbors + 1) > len(mol.GetAtoms()):
                neighbor_end_index = len(mol.GetAtoms())
            else:
                neighbor_end_index = (self.n_neighbors + 1)

            distance_atom = distance_matrix[n, :]
            cutoff_end_index = distance_atom[distance_atom < self.cutoff].size

            end_index = min(neighbor_end_index, cutoff_end_index)

            # Loop over each of the nearest neighbors

            neighbor_inds = distance_matrix[n, :].argsort()[1:end_index]
            if len(neighbor_inds)==0: neighbor_inds = [n]
            for neighbor in neighbor_inds:

                # update bond feature matrix
                bond = mol.GetBondBetweenAtoms(n, int(neighbor))
                if bond is None:
                    bond_feature_matrix[bond_index] = 0
                else:
                    rev = False if bond.GetBeginAtomIdx() == n else True
                    bond_feature_matrix[bond_index] = self.bond_tokenizer(
                        self.bond_features(bond, flipped=rev))
                    try:
                        bond_index_matrix[bond_index] = bond_index_array.tolist().index(bond.GetIdx())
                    except:
                        pass

                distance = distance_matrix[n, neighbor]
                bond_distance_matrix[bond_index] = distance

                # update connectivity matrix
                connectivity[bond_index, 0] = n
                connectivity[bond_index, 1] = neighbor

                bond_index += 1
        return {
            'n_atom': n_atom,
            'n_bond': n_bond,
            'n_pro': n_pro,
            'atom': atom_feature_matrix,
            'bond': bond_feature_matrix,
            'distance': bond_distance_matrix,
            'connectivity': connectivity,
            'bond_index': bond_index_matrix,
        }

class MolAPreprocessor(MolPreprocessor):
    """
    This is a subclass of Molpreprocessor that preprocessor molecule with
    bond property target
    """
    def __init__(self, **kwargs):
        """
        A preprocessor class that also returns bond_target_matrix, besides the bond matrix
        returned by MolPreprocessor. The bond_target_matrix is then used as ref to reduce molecule
        to bond property
        """
        super(MolAPreprocessor, self).__init__(**kwargs)

    def construct_feature_matrices(self, entry):
        """
        Given an entry contining rdkit molecule, bond_index and for the target property,
        return atom
        feature matrices, bond feature matrices, distance matrices, connectivity matrices and bond
        ref matrices.

        returns
        dict with entries
        see MolPreproccessor
        'bond_index' : ref array to the bond index
        """
        mol, atom_index_array = entry

        n_atom = len(mol.GetAtoms())
        n_pro = len(atom_index_array)

        # n_bond is actually the number of atom-atom pairs, so this is defined
        # by the number of neighbors for each atom.
        #if there is cutoff,
        distance_matrix = Chem.Get3DDistanceMatrix(mol)

        #if self.n_neighbors <= (n_atom - 1):
        #    n_bond = self.n_neighbors * n_atom
        #else:
            # If there are fewer atoms than n_neighbors, all atoms will be
            # connected
        n_bond = distance_matrix[(distance_matrix < self.cutoff) & (distance_matrix != 0)].size

        if n_bond == 0: n_bond = 1

        # Initialize the matrices to be filled in during the following loop.
        atom_feature_matrix = np.zeros(n_atom, dtype='int')
        bond_feature_matrix = np.zeros(n_bond, dtype='int')
        bond_distance_matrix = np.zeros(n_bond, dtype=np.float32)
        atom_index_matrix = np.full(n_atom, -1, dtype='int')
        connectivity = np.zeros((n_bond, 2), dtype='int')

        # Hopefully we've filtered out all problem mols by now.
        if mol is None:
            raise RuntimeError("Issue in loading mol")

        # Get a list of the atoms in the molecule.
        atom_seq = mol.GetAtoms()
        atoms = [atom_seq[i] for i in range(n_atom)]

        # Here we loop over each atom, and the inner loop iterates over each
        # neighbor of the current atom.
        bond_index = 0  # keep track of our current bond.
        for n, atom in enumerate(atoms):
            # update atom feature matrix
            atom_feature_matrix[n] = self.atom_tokenizer(
                self.atom_features(atom))
            try:
                atom_index_matrix[n] = atom_index_array.tolist().index(atom.GetIdx())
            except:
                pass
            # if n_neighbors is greater than total atoms, then each atom is a
            # neighbor.
            if (self.n_neighbors + 1) > len(mol.GetAtoms()):
                neighbor_end_index = len(mol.GetAtoms())
            else:
                neighbor_end_index = (self.n_neighbors + 1)

            distance_atom = distance_matrix[n, :]
            cutoff_end_index = distance_atom[distance_atom < self.cutoff].size

            end_index = min(neighbor_end_index, cutoff_end_index)

            # Loop over each of the nearest neighbors

            neighbor_inds = distance_matrix[n, :].argsort()[1:end_index]
            if len(neighbor_inds)==0: neighbor_inds = [n]
            for neighbor in neighbor_inds:

                # update bond feature matrix
                bond = mol.GetBondBetweenAtoms(n, int(neighbor))
                try:
                    if bond is None:
                        bond_feature_matrix[bond_index] = 0
                    else:
                        rev = False if bond.GetBeginAtomIdx() == n else True
                        bond_feature_matrix[bond_index] = self.bond_tokenizer(
                            self.bond_features(bond, flipped=rev))
                except:
                    print('AAAAAAAAAAAAAAA')
                    print(mol.GetProp('_Name'))
                    print(mol.GetProp('ConfId'))

                distance = distance_matrix[n, neighbor]
                bond_distance_matrix[bond_index] = distance

                # update connectivity matrix
                connectivity[bond_index, 0] = n
                connectivity[bond_index, 1] = neighbor

                bond_index += 1
        return {
            'n_atom': n_atom,
            'n_bond': n_bond,
            'n_pro': n_pro,
            'atom': atom_feature_matrix,
            'bond': bond_feature_matrix,
            'distance': bond_distance_matrix,
            'connectivity': connectivity,
            'atom_index': atom_index_matrix,
        }



# More Class imports from the paper (There's a lot of code review to do D:)

In [ ]:
from keras.models import Model
from keras import Layer
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.layers import Lambda
import tensorflow as tf
import keras.backend as K

class GraphModel(Model):
    """ This is a simple modification of the Keras `Model` class to avoid
    checking each input for a consistent batch_size dimension. Should work as
    of keras-team/keras#11548.

    """

    def _standardize_user_data(self, *args, **kwargs):
        kwargs['check_array_lengths'] = False
        return super(GraphModel, self)._standardize_user_data(*args, **kwargs)
class MessageLayer(Layer):
    """ Implements the matrix multiplication message functions from Gilmer
    2017. This could probably be implemented as a series of other layers, but
    this is more convenient.

    """

    def __init__(self, dropout=0., reducer=None, **kwargs):
        """

        dropout : float between 0 and 1
            Whether to apply dropout to individual messages before they are
            reduced to each incoming atom.

        reducer : ['sum', 'mean', 'max', or 'min']
            How to collect incoming messages for each atom. In this library,
            I'm careful to only have messages be a function of the sending
            atom, so we can sort the connectivity matrix by recieving atom.
            That lets us use the `segment_*` methods from tensorflow, instead
            of the `unsorted_segment_*` methods.

        """

        self.dropout = dropout
        self.reducer = reducer

        reducer_dict = {
            None: tf.segment_sum,
            'sum': tf.segment_sum,
            'mean': tf.segment_mean,
            'max': tf.segment_max,
            'min': tf.segment_min
        }

        self._reducer = reducer_dict[reducer]

        super(MessageLayer, self).__init__(**kwargs)


    def call(self, inputs, training=None):
        """ Perform a single message passing step, returing the summed messages
        for each recieving atom.

        Inputs are [atom_matrix, bond_matrix, connectivity_matrix]

        atom_matrix : (num_atoms_in_batch, d)
            The input matrix of current hidden states for each atom

        bond_matrix : (num_bonds_in_batch, d, d)
            A matrix of current edge features, with each edge represented as a
            (dxd) matrix.

        connectivity : (num_bonds_in_batch, 2)
            A matrix of (a_i, a_j) pairs that indicates the bond in bond_matrix
            connecting atom_matrix[a_j] to atom_matrix[a_i].
            The first entry indicates the recieving atom.

        """

        atom_matrix, bond_matrix, connectivity = inputs

        # Gather the atom matrix so that each reciever node corresponds with
        # the given bond_matrix entry
        atom_gathered = tf.gather(atom_matrix, connectivity[:, 1])

        # Multiply the bond matrices by the gathered atom matrices
        messages = K.batch_dot(bond_matrix, atom_gathered)

        # Add dropout on a message-by-message basis if desired
        def add_dropout():
            if 0. < self.dropout < 1.:
                return K.dropout(messages, self.dropout)
            else:
                return messages

        dropout_messages = K.in_train_phase(
            add_dropout(), messages, training=training)

        # Sum each message along the (sorted) reciever nodes
        summed_message = self._reducer(dropout_messages, connectivity[:, 0])

        return summed_message


    def compute_output_shape(self, input_shape):
        """ Computes the shape of the output, which should be the same
        dimension as the first input, that atom hidden state """

        assert input_shape and len(input_shape) == 3
        assert input_shape[0][-1]  # atom hidden state dimension must be specified
        return input_shape[0]


    def get_config(self):
        config = {
            'dropout': self.dropout,
            'reducer': self.reducer,
        }
        base_config = super(MessageLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class GatherAtomToBond(Layer):
    """ Reshapes the atom matrix (num_atoms_in_batch, d) to the bond matrix
    (num_bonds_in_batch, d) by reindexing according to which atom is involved
    in each bond.

    index : 0 or 1
        whether to gather the sending atoms (1) or recieving atoms (0) for each
        bond.

    """

    def __init__(self, index, **kwargs):
        self.index = index
        super(GatherAtomToBond, self).__init__(**kwargs)

    def call(self, inputs):
        atom_matrix, connectivity = inputs
        return  tf.gather(atom_matrix, connectivity[:, self.index])

    def compute_output_shape(self, input_shape):
        """ Computes the shape of the output,
        which should be the shape of the atom matrix with the length
        of the bond matrix """

        assert input_shape and len(input_shape) == 2
        return input_shape[0]

    def get_config(self):
        config = {
            'index': self.index,
        }
        base_config = super(GatherAtomToBond, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class Reducer(Layer):
    """ Superclass for reducing methods.

    reducer : ['sum', 'mean', 'max', or 'min']
        How to collect elements for each atom or molecule. In this library,
        I'm careful to only have messages be a function of the sending
        atom, so we can sort the connectivity matrix by recieving atom.
        That lets us use the `segment_*` methods from tensorflow, instead
        of the `unsorted_segment_*` methods.

    """

    def __init__(self, reducer=None, **kwargs):

        self.reducer = reducer

        reducer_dict = {
            None: tf.math.segment_sum,
            'sum': tf.math.segment_sum,
            'unsorted_sum': tf.math.unsorted_segment_sum,
            'mean': tf.math.segment_mean,
            'unsorted_mean': tf.math.unsorted_segment_mean,
            'max': tf.math.segment_max,
            'min': tf.math.segment_min
        }

        self._reducer = reducer_dict[reducer]

        super(Reducer, self).__init__(**kwargs)

    def get_config(self):
        config = {'reducer': self.reducer}
        base_config = super(Reducer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        # Output shape is (n_graphs, atom_dim)
        return input_shape[0]


class ReduceAtomToMol(Reducer):
    """ Sum over all atoms in each molecule.

    Inputs

    atom_matrix : (num_atoms_in_batch, d)
        atom hidden states for each atom in the batch

    node_graph_indices : (num_atoms_in_batch,)
        A scalar for each atom representing which molecule in the batch the
        atom belongs to. This is generated by the preprocessor class, and
        essentially looks like [0, 0, 0, 1, 1] for a batch with a 3 atom
        molecule and a 2 atom molecule.
    """

    def call(self, inputs):

        atom_matrix, node_graph_indices = inputs
        return self._reducer(atom_matrix, node_graph_indices)

class ReduceBondToPro(Reducer):
    """
    Sums over bonds acoording to bond_index to get target bond properties

    Inputs

    bond_matrix : (num_bonds_in_batch, d)
        bond hidden states for each bond in the batch

    bond_index : (bond_atoms_in_batch, )
        A scalar for each bond representing which number in the target property
        the bond links to. This is generated by the preprocessor class, and
        essentially looks like [-1,-1,0,0,-1,-1,1....]
    """

    def call(self, inputs):

        bond_matrix, bond_index, n_pro = inputs
        num_segments = tf.reduce_sum(n_pro)
        return self._reducer(bond_matrix, bond_index, num_segments)

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 3
        return input_shape[0]

class ReduceAtomToPro(Reducer):
    """
    Sums over atoms acoording to atom_index to get target atom properties

    Inputs

    atom_matrix : (num_atoms_in_batch, d)
        atom hidden states for each atom in the batch

    atom_index : (atom_atoms_in_batch, )
        A scalar for each atom representing which number in the target property
        the atom links to. This is generated by the preprocessor class, and
        essentially looks like [-1,-1,0,0,-1,-1,1....]
    """

    def call(self, inputs):

        atom_matrix, atom_index, n_pro = inputs
        num_segments = tf.reduce_sum(n_pro)
        return self._reducer(atom_matrix, atom_index, num_segments)

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 3
        return input_shape[0]

class ReduceBondToAtom(Reducer):

    """ Sums over the incoming messages from all sender atoms.

    Inputs:

    bond_matrix : (num_bonds_in_batch, d)
        A matrix of messages coming from each sender atom; one row for each
        bond/edge.

    connectivity : (num_bonds_in_batch, 2)
        A matrix of (a_i, a_j) pairs that indicates the bond in bond_matrix
        connecting atom_matrix[a_j] to atom_matrix[a_i].
        The first entry indicates the recieving atom.

    Again, I'm careful to only have the messages be a function of the sending
    node, such that we can use sorted methods in performing the reduction.

    """

    def call(self, inputs):

        bond_matrix, connectivity = inputs
        return self._reducer(bond_matrix, connectivity[:, 0])


class Squeeze(Layer):
    """ Keras forces inputs to be a vector per entry, so this layer squeezes
    them to a single dimension.

    I.e., node_graph_indices will have shape (num_atoms_in_batch, 1), while its
    easier to work with a vector of shape (num_atoms_in_batch,)
    """

    def call(self, inputs):
        return K.squeeze(inputs, 1)

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]


class Embedding2D(Layer):
    """ Keras typically wants to embed items as a single vector, while for the
    matrix multiplication method of Gilmer 2017 we need a matrix for each bond
    type. This just implements that fairly simple extension of the traditional
    embedding layer.
    """

    def __init__(self, input_dim, output_dim,
                 embeddings_initializer='uniform',
                 embeddings_regularizer=None,
                 embeddings_constraint=None,
                 **kwargs):

        super(Embedding2D, self).__init__(**kwargs)

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.embeddings_initializer = initializers.get(embeddings_initializer)
        self.embeddings_regularizer = regularizers.get(embeddings_regularizer)
        self.embeddings_constraint = constraints.get(embeddings_constraint)

    def build(self, input_shape):

        self.embeddings = self.add_weight(
            shape=(self.input_dim, self.output_dim, self.output_dim),
            initializer=self.embeddings_initializer,
            name='bond_embedding_weights',
            regularizer=self.embeddings_regularizer,
            constraint=self.embeddings_constraint)

        self.built = True

    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim, self.output_dim)

    def get_config(self):
        config = {
            'input_dim': self.input_dim,
            'output_dim': self.output_dim,
            'embeddings_initializer':
            initializers.serialize(self.embeddings_initializer),
            'embeddings_regularizer':
            regularizers.serialize(self.embeddings_regularizer),
            'embeddings_constraint':
            constraints.serialize(self.embeddings_constraint),
        }
        base_config = super(Embedding2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class EdgeNetwork(Layer):
    """ A layer to embed (bond_type, distance) pairs as a NxN matrix.

    Inputs:
    units : dimension of the output matrix
    bond_classes : number of unique bonds

    First perfoms a 1-hot encoding of the bond_type, then passes the
    (*one_hot_encoding, distance) vector to a dense layer. This is the "Edge
    Network" message described by Gilmer, 2017.

    """

    def __init__(self, units, bond_classes,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        super(EdgeNetwork, self).__init__(**kwargs)
        self.units = units
        self.bond_classes = bond_classes

        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

    def build(self, input_shape):

        self.kernel = self.add_weight(shape=(self.bond_classes + 1, self.units**2),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units**2,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, inputs):

        bond_type, distance = inputs
        bond_type_onehot = tf.one_hot(tf.squeeze(bond_type), self.bond_classes)
        stacked_inputs = tf.concat([bond_type_onehot, distance], 1)

        output = K.dot(stacked_inputs, self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)

        output = tf.reshape(output, [-1, self.units, self.units])
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.units, self.units)

    def get_config(self):
        config = {
            'units': self.units,
            'bond_classes': self.bond_classes,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer':
                regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(EdgeNetwork, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
train_file = '/content/drive/MyDrive/CSC461/train.pkl.gz'
test_file = '/content/drive/MyDrive/CSC461/test.pkl.gz'
val_file = '/content/drive/MyDrive/CSC461/valid.pkl.gz'
train = pd.read_pickle(train_file)
test = pd.read_pickle(test_file)
val = pd.read_pickle(val_file)

In [ ]:
y_train = train.Shift.values
y_test = test.Shift.values
y_val = val.Shift.values
y_train[0]

array([ 6.2862277,  1.8515768,  3.06077  ,  1.8287755,  2.1147938,
        1.9405926,  2.4125774,  5.2048125,  1.7113038,  1.9827293,
        1.6478252,  5.1483564,  9.646486 ,  6.1986713,  2.1315756,
        2.0812304,  2.3436267,  1.7480594,  2.5291371,  1.9526317,
        5.372994 ,  1.7547172,  1.7854533,  2.181191 , 11.188395 ,
        6.192834 ], dtype=float32)

In [ ]:
processed_inputs_path = '/content/drive/MyDrive/CSC461/processed_inputs.p'

# Load the pickle file
with open(processed_inputs_path, 'rb') as file:
    processed_inputs = pickle.load(file)

# Access the elements in the dictionary
inputs_train = processed_inputs['inputs_train']
inputs_valid = processed_inputs['inputs_valid']
inputs_test = processed_inputs['inputs_test']
preprocessor = processed_inputs['preprocessor']

inputs_train[0]

{'n_atom': 57,
 'n_bond': 1274,
 'n_pro': 26,
 'atom': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 'bond': array([2, 3, 3, ..., 0, 0, 0]),
 'distance': array([1.0817016, 1.3787208, 1.3937682, ..., 4.7569475, 4.868445 ,
        4.896678 ], dtype=float32),
 'connectivity': array([[ 0, 31],
        [ 0,  5],
        [ 0,  1],
        ...,
        [56, 53],
        [56, 25],
        [56, 54]]),
 'atom_index': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  1,  2,
         3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25])}

In [ ]:
with open(processed_inputs_path, 'rb') as f:
    input_data = pickle.load(f)

preprocessor = input_data['preprocessor']

# More functions

In [ ]:
def rbf_expansion(distances, mu=0, delta=0.1, kmax=256):
    k = np.arange(0, kmax)
    logits = -(np.atleast_2d(distances).T - (-mu + delta * k))**2 / delta
    return np.exp(logits)

def atomic_number_tokenizer(atom):
    return atom.GetNumRadicalElectrons()

def _compute_stacked_offsets(sizes, repeats):
    return np.repeat(np.cumsum(np.hstack([0, sizes[:-1]])), repeats)

In [ ]:
import numpy as np
from random import shuffle

from keras.utils import Sequence

class GraphSequence(Sequence):

    def __init__(self, inputs, y=None, batch_size=1, shuffle=True,
                 final_batch=True):
        """ A keras.Sequence generator to be passed to model.fit_generator. (or
        any other *_generator method.) Returns (inputs, y) tuples where
        molecule feature matrices have been stitched together. Offsets the
        connectivity matrices such that atoms are indexed appropriately.

        batch_size: number of molecules per batch
        shuffle : whether to shuffle the input data
        final_batch : whether to include the final, incomplete batch

        """
        self._inputs = inputs
        self._y = np.asarray(y) if y is not None else None
        self._input_keys = list(inputs[0].keys())
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.final_batch = final_batch


    def __len__(self):
        """ Total number of batches """
        if self.final_batch:
            return int(np.ceil(len(self._inputs) / float(self.batch_size)))
        else:
            return int(np.floor(len(self._inputs) / float(self.batch_size)))

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(0, len(self._inputs))
            np.random.shuffle(indices)
            self._inputs = [self._inputs[i] for i in indices]
            if self._y is not None:
                self._y = self._y[indices]

    def __getitem__(self, idx):
        """ Calculate the feature matrices for a whole batch (with index `i` <
        self.__len__). This involves adding offsets to the indices for each
        atom in the connectivity matrix; such that atoms and bonds in later
        molecules still refer to the correct atoms.

        """
        batch_indexes = idx * self.batch_size + np.arange(0, self.batch_size)
        batch_indexes = batch_indexes[batch_indexes < len(self._inputs)]

        batch_data = {
            key: self._concat([self._inputs[i][key] for i in batch_indexes])
            for key in self._input_keys}

        # Offset the connectivity matrix to account for the multiple graphs per
        # batch
        offset = _compute_stacked_offsets(
            batch_data['n_atom'], batch_data['n_bond'])

        batch_data['connectivity'] += offset[:, np.newaxis]

        # Compute graph indices with shape (n_atom,) that indicate to which
        # molecule each atom belongs.
        n_graphs = len(batch_indexes)
        batch_data['node_graph_indices'] = np.repeat(
            np.arange(n_graphs), batch_data['n_atom'])

        batch_data = self.process_data(batch_data)

        # Keras takes to options, one (x, y) pairs, or just (x,) pairs if we're
        # doing predictions. Here, if we've specified a y matrix, we return the
        # x,y pairs for training, otherwise just return the x data.
        if self._y is not None:
            return (batch_data, np.concatenate(self._y[batch_indexes]).reshape(-1,1))

        else:
            return batch_data

    def process_data(self, batch_data):
        """ function to add additional processing to batch data before returning """

        # These aren't used currently, so I pop them. But we might need them at
        # a later time.
        del batch_data['n_atom']
        del batch_data['n_bond']

        return batch_data


    def _concat(self, to_stack):
        """ function to stack (or concatentate) depending on dimensions """

        if np.asarray(to_stack[0]).ndim >= 2:
            return np.concatenate(to_stack)

        else:
            return np.hstack(to_stack)


def _compute_stacked_offsets(sizes, repeats):
    """ Computes offsets to add to indices of stacked np arrays.
    When a set of np arrays are stacked, the indices of those from the second on
    must be offset in order to be able to index into the stacked np array. This
    computes those offsets.

    Args:
        sizes: A 1D sequence of np arrays of the sizes per graph.
        repeats: A 1D sequence of np arrays of the number of repeats per graph.
    Returns:
        The index offset per graph.
    """
    return np.repeat(np.cumsum(np.hstack([0, sizes[:-1]])), repeats)

class RBFSequence(GraphSequence):
    def process_data(self, batch_data):
        batch_data['distance_rbf'] = rbf_expansion(batch_data['distance'])

        offset = _compute_stacked_offsets(
            batch_data['n_pro'], batch_data['n_atom'])

        offset = np.where(batch_data['atom_index']>=0, offset, 0)
        batch_data['atom_index'] += offset

        del batch_data['n_atom']
        del batch_data['n_bond']
        del batch_data['distance']

        return batch_data

In [ ]:
X = []
Y = []
for row,y in zip(input_data['inputs_train'], y_train):
    X.extend(row['atom'][row['atom_index']>=0])
    Y.extend(y[row['atom_index'][row['atom_index']>=0]])

In [ ]:
atom_total = 0
for mol in inputs_train:
  atom_total += mol['n_pro']
atom_total

102199

In [ ]:
input_data['inputs_train'][0]
y_train[0]

array([ 6.2862277,  1.8515768,  3.06077  ,  1.8287755,  2.1147938,
        1.9405926,  2.4125774,  5.2048125,  1.7113038,  1.9827293,
        1.6478252,  5.1483564,  9.646486 ,  6.1986713,  2.1315756,
        2.0812304,  2.3436267,  1.7480594,  2.5291371,  1.9526317,
        5.372994 ,  1.7547172,  1.7854533,  2.181191 , 11.188395 ,
        6.192834 ], dtype=float32)

In [ ]:
Y[1]

1.8515768

In [ ]:
X[0]

4

In [ ]:
inputs_train[0]

{'n_atom': 57,
 'n_bond': 1274,
 'n_pro': 26,
 'atom': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 'bond': array([2, 3, 3, ..., 0, 0, 0]),
 'distance': array([1.0817016, 1.3787208, 1.3937682, ..., 4.7569475, 4.868445 ,
        4.896678 ], dtype=float32),
 'connectivity': array([[ 0, 31],
        [ 0,  5],
        [ 0,  1],
        ...,
        [56, 53],
        [56, 25],
        [56, 54]]),
 'atom_index': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  1,  2,
         3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25])}

In [ ]:
atom_means = pd.DataFrame({'atom':X, 'shift':Y}).dropna().groupby('atom')['shift'].mean()

In [ ]:
atom_means = atom_means.reindex(np.arange(preprocessor.atom_classes)).fillna(0)
atom_means

,shift
atom,
0,0.000000
1,0.000000
2,3.022405
3,3.280265
4,3.952668
5,0.000000
6,0.000000
7,1.258198
8,0.000000


In [ ]:
pd.DataFrame({'atom':X, 'shift':Y})['atom'].value_counts()

,count
atom,
4,102154
2,31
3,13
7,1


In [ ]:
batch_size = 32
train_sequence = RBFSequence(input_data['inputs_train'], y_train, batch_size)
valid_sequence = RBFSequence(input_data['inputs_valid'], y_val, batch_size)

In [ ]:
dir(train_sequence)
train_sequence.__class__

__main__.RBFSequence

In [ ]:
batch_data = input_data['inputs_train'][0]

batch_data['distance_rbf'] = rbf_expansion(batch_data['distance'])
# offset = _compute_stacked_offsets(batch_data['n_pro'], batch_data['n_atom'])
# sizes = batch_data['n_atom']
# repeats = batch_data['n_bond']
# np.repeat(np.cumsum(np.hstack([0, sizes[:-1]])), repeats)
batch_data

{'n_atom': 57,
 'n_bond': 1274,
 'n_pro': 26,
 'atom': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 3, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 'bond': array([2, 3, 3, ..., 0, 0, 0]),
 'distance': array([1.0817016, 1.3787208, 1.3937682, ..., 4.7569475, 4.868445 ,
        4.896678 ], dtype=float32),
 'connectivity': array([[ 0, 31],
        [ 0,  5],
        [ 0,  1],
        ...,
        [56, 53],
        [56, 25],
        [56, 54]]),
 'atom_index': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  1,  2,
         3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25]),
 'distance_rbf': array([[8.28731687e-006, 6.52437027e-005, 4.20537142e-004, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [5.55421160e-009, 7.9201719

In [ ]:
len(train_sequence[2][0]['n_pro'])

32

# IMPORTANT

In [ ]:
train_sequence[2][0]

{'n_pro': array([23, 33, 22, 24, 16, 20, 20, 29, 31,  4, 26, 21, 30, 20, 22, 22, 21,
         7, 13, 28, 39, 20, 36, 11, 24, 18, 14, 30, 40, 16, 27, 44]),
 'atom': array([2, 2, 2, ..., 4, 4, 4]),
 'bond': array([2, 2, 2, ..., 0, 0, 0]),
 'connectivity': array([[   0,   22],
        [   0,   23],
        [   0,   24],
        ...,
        [1500, 1445],
        [1500, 1492],
        [1500, 1447]]),
 'atom_index': array([ -1,  -1,  -1, ..., 748, 749, 750]),
 'node_graph_indices': array([ 0,  0,  0, ..., 31, 31, 31]),
 'distance_rbf': array([[7.14868225e-006, 5.70513607e-005, 3.72775296e-004, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [6.49087834e-006, 5.22621210e-005, 3.44517825e-004, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [6.45558713e-006, 5.20039009e-005, 3.42986638e-004, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        ...,
        [1.92554872e-098, 2.29713828e-094, 2.24367976e-090, ...,
        

# IMPORTANT

In [ ]:
atom_index = Input(shape=(1,), name='atom_index', dtype='int32')
atom_types = Input(shape=(1,), name='atom', dtype='int32')
distance_rbf = Input(shape=(256,), name='distance_rbf', dtype='float32')
connectivity = Input(shape=(2,), name='connectivity', dtype='int32')
n_pro = Input(shape=(1,), name='n_pro', dtype='int32')

In [ ]:
atom_index

<KerasTensor shape=(None, 1), dtype=int32, sparse=False, name=atom_index>

# IMPORTANT

In [ ]:
squeeze = Squeeze()

satom_index = squeeze(atom_index)
satom_types = squeeze(atom_types)
sn_pro = squeeze(n_pro)
# Initialize RNN and MessageLayer instances
atom_features = 256

# Initialize the atom states
atom_state = Embedding(
    preprocessor.atom_classes,
    atom_features, name='atom_embedding')(satom_types)

atomwise_shift = Embedding(
    preprocessor.atom_classes, 1, name='atomwise_shift',
    embeddings_initializer=keras.initializers.constant(atom_means.values)
)(satom_types)

bond_state = distance_rbf

# IMPORTANT

In [ ]:
def message_block(atom_state, bond_state, connectivity):

    atom_state = Dense(atom_features, use_bias=False)(atom_state)

    source_atom_gather = GatherAtomToBond(1)
    target_atom_gather = GatherAtomToBond(0)

    source_atom = source_atom_gather([atom_state, connectivity])
    target_atom = target_atom_gather([atom_state, connectivity])

    # Edge update network
    bond_state_message = Concatenate()([source_atom, target_atom, bond_state])
    bond_state_message = Dense(2*atom_features, activation='softplus')(bond_state_message)
    bond_state_message = Dense(atom_features)(bond_state_message)

    bond_state_message = Dense(atom_features, activation='softplus')(bond_state_message)
    bond_state_message = Dense(atom_features, activation='softplus')(bond_state_message)
    bond_state = Add()([bond_state_message, bond_state])

    # message function
    messages = Multiply()([source_atom, bond_state])
    messages = ReduceBondToAtom(reducer='sum')([messages, connectivity])

    # state transition function
    messages = Dense(atom_features, activation='softplus')(messages)
    messages = Dense(atom_features)(messages)
    atom_state = Add()([atom_state, messages])

    return atom_state, bond_state

In [ ]:
bond_state

<KerasTensor shape=(None, 256), dtype=float32, sparse=False, name=distance_rbf>

# IMPORTANT

In [ ]:
for _ in range(3):
    atom_state, bond_state = message_block(atom_state, bond_state, connectivity)

# Going through test forward pass of message passing in the CASCADE model

# IMPORTANT (INSPECTION OF MESSAGE BLOCK FUNCTION)

In [ ]:
#Testing Keras Embedding Layer to see whats happening (go from atom label to 256 vector)
test_atom_state = Embedding(
    preprocessor.atom_classes,
    atom_features, name='atom_embedding')
model = keras.Sequential()
model.add(test_atom_state)
input = np.array([2,2])
test_atom_vectors = model.predict(input)
#Defining connectivity, bond state for a simple one connection molecule
test_connectivity = tf.convert_to_tensor(np.array(([0,1],[1,0])))
test_bond_state = rbf_expansion(np.array([5,5]))
#One Dense layer is run on the atoms initially
test_atom_vectors = Dense(atom_features, use_bias=False)(test_atom_vectors)
#Gather target and source atoms for messaging
gather_thingy = GatherAtomToBond(0)
other_gather_thingy = GatherAtomToBond(1)
target_atoms = gather_thingy(inputs = [test_atom_vectors,test_connectivity])
source_atoms = other_gather_thingy(inputs = [test_atom_vectors,test_connectivity])
#Concatenates target, source and bond vectors into one long vector (3*256 in this case)
bond_message = Concatenate()([target_atoms, source_atoms, test_bond_state])
#Dense layer to 512 length vector
bond_message = Dense(2*atom_features, activation='softplus')(bond_message)
#Dense layer back to 256 length vector
bond_message = Dense(atom_features)(bond_message)
# Adding the message to the existing bond value and getting updated value
test_bond_state = Add()([bond_message, test_bond_state])
# Working on updating nodes, starting with element wise multiplication of the source atoms and the edges
node_message = Multiply()([source_atoms, test_bond_state])
# Adds all bonds connecting a single atom together
node_message = ReduceBondToAtom(reducer='sum')([node_message, connectivity])
# Two dense layers
node_message = Dense(atom_features, activation='softplus')(node_message)
node_message = Dense(atom_features)(node_message)
# Adding the node messages to get the final node values
final_atom_vectors = Add()([atom_state, node_message])
final_atom_vectors.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step


(2, 256)

# IMPORTANT

In [ ]:
atom_state = ReduceAtomToPro(reducer='unsorted_mean')([atom_state, satom_index, sn_pro])
atomwise_shift = ReduceAtomToPro(reducer='unsorted_mean')([atomwise_shift, satom_index, sn_pro])

atom_state = Dense(atom_features, activation='softplus')(atom_state)
atom_state = Dense(atom_features, activation='softplus')(atom_state)
atom_state = Dense(atom_features//2, activation='softplus')(atom_state)
atom_state = Dense(1)(atom_state)

output = Add()([atom_state, atomwise_shift])


In [ ]:
atom_index

<KerasTensor shape=(None, 1), dtype=int32, sparse=False, name=atom_index>

# IMPORTANT

In [ ]:
lr = 5E-4
epochs = 1200

model = GraphModel([
        atom_index, atom_types, distance_rbf, connectivity, n_pro], [atom_index])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss='mae')

model.summary()

Model: "graph_model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ atom (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ distance_rbf (InputLayer) │ (None, 256)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ connectivity (InputLayer) │ (None, 2)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ n_pro (InputLayer)        │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ atom_index (InputLayer)   │ (None, 1)              │              0 │ -                      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# IMPORTANT

In [ ]:
def decay_fn(epoch, learning_rate):
    """ Jorgensen decays to 0.96*lr every 100,000 batches, which is approx
    every 28 epochs """
    if (epoch % 70) == 0:
        return 0.96 * learning_rate
    else:
        return learning_rate

lr_decay = LearningRateScheduler(decay_fn)

# IMPORTANT

In [ ]:
filepath = "/content/drive/MyDrive/CSC461/best_model.keras"
csv_logger = CSVLogger('/content/drive/MyDrive/CSC461/log.csv')
checkpoint = ModelCheckpoint(filepath, save_best_only=True, save_freq='epoch', verbose=1)
hist = model.fit(train_sequence, validation_data=valid_sequence,
                           epochs=epochs, verbose=1,
                           callbacks=[checkpoint, csv_logger, lr_decay])

Epoch 1/1200


ValueError: Exception encountered when calling GraphModel.call().

[1mInvalid input shape for input Tensor("graph_model_2_1/Cast_2:0", shape=(None,), dtype=float32). Expected shape (None, 256), but input has incompatible shape (None,)[0m

Arguments received by GraphModel.call():
  • inputs={'n_pro': 'tf.Tensor(shape=(None,), dtype=int64)', 'atom': 'tf.Tensor(shape=(None,), dtype=int64)', 'bond': 'tf.Tensor(shape=(None,), dtype=int64)', 'connectivity': 'tf.Tensor(shape=(None, 2), dtype=int64)', 'atom_index': 'tf.Tensor(shape=(None,), dtype=int64)', 'node_graph_indices': 'tf.Tensor(shape=(None,), dtype=int64)', 'distance_rbf': 'tf.Tensor(shape=(None, 256), dtype=float64)'}
  • training=True
  • mask={'n_pro': 'None', 'atom': 'None', 'bond': 'None', 'connectivity': 'None', 'atom_index': 'None', 'node_graph_indices': 'None', 'distance_rbf': 'None'}

In [ ]:
valid_sequence[0][0]['distance_rbf'].shape

(16770, 256)

In [ ]:
test_mod = keras.Sequential()
test_mod.add(Input(shape=(256,), name='distance_rbf', dtype='float32'))
test_mod.add(Dense(256, activation='softplus'))
test_mod.summary()
test_mod.predict(train_sequence[0][0]['distance_rbf'])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,792 (257.00 KB)

 Trainable params: 65,792 (257.00 KB)

 Non-trainable params: 0 (0.00 B)

1037/1037 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step


array([[0.75749016, 0.57033974, 0.67821103, ..., 0.66581386, 0.7193932 ,
        0.80530006],
       [0.7468433 , 0.56934005, 0.6820942 , ..., 0.63832223, 0.77921623,
        0.79469776],
       [0.7437823 , 0.57471913, 0.6855024 , ..., 0.64088094, 0.78454655,
        0.79514563],
       ...,
       [0.6949312 , 0.6935917 , 0.6204102 , ..., 0.6970189 , 0.6164915 ,
        0.6758731 ],
       [0.68610764, 0.6947843 , 0.6158033 , ..., 0.70515215, 0.6292391 ,
        0.6921318 ],
       [0.6856468 , 0.6948941 , 0.61546797, ..., 0.7058479 , 0.63004595,
        0.69348747]], dtype=float32)

In [ ]:
train_sequence[0][0]

NameError: name 'train_sequence' is not defined